# Tune single wake

Use the 8 m/s two turbine aligned case to find better tunings for new gauss model

In [48]:
import matplotlib.pyplot as plt
import floris.tools as wfct
import numpy as np
import pandas as pd
import copy
import seaborn as sns
import pickle
from scipy.optimize import minimize, basinhopping

## Load FLORIS models and SOWFA DATA

In [4]:
df = pickle.load( open( "sowfa_04.p", "rb" ) )
fi_dict = pickle.load( open( "floris_models_2.p", "rb" ) )

In [5]:
# Make a color dict
color_dict = dict()
for floris_key in fi_dict:
    (fi, floris_color, floris_marker, floris_label) = fi_dict[floris_key]
    color_dict[floris_label] = floris_color

## Limit DF to 2 turbine aligned cases

In [6]:
# Limit to aligned only
print(df.shape)
df = df[df.yaw.apply(lambda x: np.max(np.abs(x)))==0.0]
# Limit to over 2-turbine
df = df[df.num_turbines>2]
df.shape

(679, 21)


(38, 21)

In [7]:
# Limit to 8 m/s
df = df[df.sowfa_U0==8.]
df.shape

(36, 21)

In [8]:
df.head()

,Precursor,case_name,floris_TI,floris_U0,layout_x,layout_y,num_turbines,path,sim_length,sowfa_TI,...,yaw,power,ti,d_spacing,wake_table,freestream_gauss,freestream_gauss_tune,freestream_gch_paper,freestream_legacy,freestream_ti
casename,,,,,,,,,,,,,,,,,,,,,
c_43,neutral_5kmx2kmx1km\n,C_0001_0_0_5_-1_-1,0.09,8.38,"(1000.0, 1630.0, 2260.0)","(1000.0, 937.0, 937.0)",3,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,...,"[0.0, 0.0, 0.0]","[1936.0, 1064.6, 637.4]","[0.09295820276858346, 0.18046822286421585, 0.1...",5.0,"[0, 1, 2]",8.370503,8.392688,8.370503,8.370503,0.092958
c_47,neutral_5kmx2kmx1km\n,C_0002_0_0_5_-1_0,0.09,8.38,"(1000.0, 1630.0, 2260.0)","(1000.0, 937.0, 1000.0)",3,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,...,"[0.0, 0.0, 0.0]","[1936.2, 1063.6, 787.2]","[0.0929894722241254, 0.18059565754881532, 0.20...",5.0,"[0, 1, 2]",8.370791,8.392977,8.370791,8.370791,0.092989
c_51,neutral_5kmx2kmx1km\n,C_0003_0_0_5_-1_1,0.09,8.38,"(1000.0, 1630.0, 2260.0)","(1000.0, 937.0, 1063.0)",3,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,...,"[0.0, 0.0, 0.0]","[1936.5, 1062.8, 1249.3]","[0.09300642801911037, 0.18063546230231642, 0.1...",5.0,"[0, 1, 2]",8.371224,8.393411,8.371224,8.371224,0.093006
c_67,neutral_5kmx2kmx1km\n,D_0073_0_0_5_0_-1,0.09,8.38,"(1000.0, 1630.0, 2260.0)","(1000.0, 1000.0, 937.0)",3,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,...,"[0.0, 0.0, 0.0]","[1936.5, 745.7, 945.4]","[0.09288084253201641, 0.10409162766590484, 0.1...",5.0,"[0, 1, 2]",8.371224,8.393411,8.371224,8.371224,0.092881
c_92,neutral_5kmx2kmx1km\n,D_0074_0_0_5_0_0,0.09,8.38,"(1000.0, 1630.0, 2260.0)","(1000.0, 1000.0, 1000.0)",3,sowfa_library (Paul.fleming@nrel.gov)/full_run...,2000.0,0.1,...,"[0.0, 0.0, 0.0]","[1936.6, 746.1, 737.1]","[0.09289228315365378, 0.1040844378798465, 0.18...",5.0,"[0, 1, 2]",8.371368,8.393556,8.371368,8.371368,0.092892


## Try to find an improved tuning for ti parameters

In [9]:
# Define a tuning error function
(fi_tune, floris_color, floris_marker, floris_label) = fi_dict['gt']

model_params = fi_tune.get_model_parameters()
model_params

{'Wake Velocity Parameters': {'alpha': 0.58,
  'beta': 0.077,
  'calculate_VW_velocities': True,
  'eps_gain': 0.3,
  'ka': 0.22077999999999998,
  'kb': 0.002,
  'use_yaw_added_recovery': True,
  'yaw_recovery_alpha': 0.03},
 'Wake Deflection Parameters': {'ad': 0.0,
  'alpha': 0.58,
  'bd': 0.0,
  'beta': 0.077,
  'eps_gain': 0.3,
  'ka': 0.38,
  'kb': 0.004,
  'use_secondary_steering': True},
 'Wake Turbulence Parameters': {'ti_ai': 0.8,
  'ti_constant': 0.73,
  'ti_downstream': -0.275,
  'ti_initial': 0.1}}

In [50]:
# Define a tuning error function
(fi_tune, floris_color, floris_marker, floris_label) = fi_dict['gt']

model_params = fi_tune.get_model_parameters()


def tuning_error_function(x): #where x = [ka, kb]
    
    error_values = np.zeros(1000)
    
    # Change the parameters
    model_params['Wake Turbulence Parameters']['ti_initial'] = x[0] * 0.1 # how much to scale inital by
    model_params['Wake Turbulence Parameters']['ti_constant'] = x[1] * 0.73 # how much to scale inital by
    model_params['Wake Turbulence Parameters']['ti_ai'] = x[2] * 0.8 # how much to scale inital by
    model_params['Wake Turbulence Parameters']['ti_downstream'] = x[3] * -0.275 # how much to scale inital by
    

    fi_tune.set_model_parameters(model_params,verbose=False)
    
    # Get the powers for each case

    count = 0
    for idx, (i, row) in enumerate(df.iterrows()):

        # Determine free-stream wind speed column
        freestream_name = 'freestream_gauss_tune'
        
        # Match the layout

        fi_tune.reinitialize_flow_field(layout_array=[row.layout_x,row.layout_y],
                                   wind_speed=[row[freestream_name]],
                                  turbulence_intensity=[row.freestream_ti])


        fi_tune.calculate_wake()
        
        floris_power = np.array(fi_tune.get_turbine_power())/1000.
        
        # Now sort out the powers by wakes
        upstream_idx = np.where(row.wake_table==0)
        sowfa_upstream_power = np.mean(row.power[upstream_idx])
        floris_upstream_power = np.mean(floris_power[upstream_idx])

        mult_wake_indices = np.where(row.wake_table>1)
        # print(mult_wake_indices,row.layout_x,floris_power)
        for m_idx in mult_wake_indices[0]:
            sowfa_downstream_power = np.mean(row.power[m_idx])
            floris_downstream_power = np.mean(floris_power[m_idx])
            sowfa_wake_loss = sowfa_upstream_power - sowfa_downstream_power
            floris_wake_loss = floris_upstream_power - floris_downstream_power
            error_values[count] = floris_wake_loss - sowfa_wake_loss
            count = count + 1
    # Return the RMS error
    error_values = error_values[:count]
    rms = np.sqrt(np.sum((error_values**2))/count)
    print('%.1f' % rms,'%.3f' % (x[0] * 0.1),'%.3f' % (x[1] * 0.73),'%.3f' % (x[2] * 0.8),'%.3f' %  (x[3] * -.275),x)
    return rms
        
        
    

In [54]:
# bnds = ((0.1,10.), (0.1,10.), (0.1,10.), (0.1,10.))# your bounds
bound = 2.
bnds = [(1/bound,bound) for i in range(4)]

minimizer_kwargs = { "method": "L-BFGS-B","bounds":bnds }

result = basinhopping(tuning_error_function, x0=[1, 1, 1, 1],minimizer_kwargs=minimizer_kwargs)

104.5 0.100 0.730 0.800 -0.275 [1. 1. 1. 1.]
104.5 0.100 0.730 0.800 -0.275 [1.00000001 1.         1.         1.        ]
104.5 0.100 0.730 0.800 -0.275 [1.         1.00000001 1.         1.        ]
104.5 0.100 0.730 0.800 -0.275 [1.         1.         1.00000001 1.        ]
104.5 0.100 0.730 0.800 -0.275 [1.         1.         1.         1.00000001]
153.9 0.500 0.146 4.000 -1.375 [5.  0.2 5.  5. ]
153.9 0.500 0.146 4.000 -1.375 [5.00000001 0.2        5.         5.        ]
153.9 0.500 0.146 4.000 -1.375 [5.         0.20000001 5.         5.        ]
153.9 0.500 0.146 4.000 -1.375 [5.         0.2        5.00000001 5.        ]
153.9 0.500 0.146 4.000 -1.375 [5.         0.2        5.         5.00000001]
152.2 0.219 0.556 1.751 -0.602 [2.18836601 0.7623268  2.18836601 2.18836601]
152.2 0.219 0.556 1.751 -0.602 [2.18836602 0.7623268  2.18836601 2.18836601]
152.2 0.219 0.556 1.751 -0.602 [2.18836601 0.76232681 2.18836601 2.18836601]
152.2 0.219 0.556 1.751 -0.602 [2.18836601 0.7623268  2.188

103.4 0.020 0.156 0.281 -0.220 [0.2        0.21427195 0.35105547 0.80000229]
91.7 0.026 0.297 0.481 -0.248 [0.26122362 0.40639022 0.60160789 0.90301676]
91.7 0.026 0.297 0.481 -0.248 [0.26122363 0.40639022 0.60160789 0.90301676]
91.7 0.026 0.297 0.481 -0.248 [0.26122362 0.40639023 0.60160789 0.90301676]
91.7 0.026 0.297 0.481 -0.248 [0.26122362 0.40639022 0.6016079  0.90301676]
91.7 0.026 0.297 0.481 -0.248 [0.26122362 0.40639022 0.60160789 0.90301677]
96.5 0.020 0.189 0.322 -0.226 [0.2        0.25952025 0.40285899 0.82197881]
96.5 0.020 0.189 0.322 -0.226 [0.20000001 0.25952025 0.40285899 0.82197881]
96.5 0.020 0.189 0.322 -0.226 [0.2        0.25952026 0.40285899 0.82197881]
96.5 0.020 0.189 0.322 -0.226 [0.2        0.25952025 0.402859   0.82197881]
96.5 0.020 0.189 0.322 -0.226 [0.2        0.25952025 0.40285899 0.82197882]
91.7 0.026 0.294 0.477 -0.248 [0.25952106 0.40230592 0.59608089 0.90076318]
91.7 0.026 0.294 0.477 -0.248 [0.25952107 0.40230592 0.59608089 0.90076318]
91.7 0.026 

91.6 0.020 0.194 0.341 -0.128 [0.2        0.26532838 0.42574727 0.46374372]
91.6 0.020 0.194 0.341 -0.128 [0.2        0.26532838 0.42574726 0.46374373]
92.2 0.020 0.146 0.160 -0.138 [0.2        0.2        0.2        0.50288401]
92.2 0.020 0.146 0.160 -0.138 [0.20000001 0.2        0.2        0.50288401]
92.2 0.020 0.146 0.160 -0.138 [0.2        0.20000001 0.2        0.50288401]
92.2 0.020 0.146 0.160 -0.138 [0.2        0.2        0.20000001 0.50288401]
92.2 0.020 0.146 0.160 -0.138 [0.2        0.2        0.2        0.50288402]
91.6 0.020 0.187 0.317 -0.129 [0.2        0.25662257 0.39566367 0.46895964]
91.6 0.020 0.187 0.317 -0.129 [0.20000001 0.25662257 0.39566367 0.46895964]
91.6 0.020 0.187 0.317 -0.129 [0.2        0.25662258 0.39566367 0.46895964]
91.6 0.020 0.187 0.317 -0.129 [0.2        0.25662257 0.39566368 0.46895964]
91.6 0.020 0.187 0.317 -0.129 [0.2        0.25662257 0.39566367 0.46895965]
91.5 0.020 0.165 0.212 -0.135 [0.2        0.22573799 0.26504567 0.49265978]
91.5 0.020 0

152.8 0.180 0.271 1.394 -0.650 [1.79844341 0.3708431  1.74273    2.3640557 ]
152.8 0.180 0.271 1.394 -0.650 [1.79844341 0.37084309 1.74273001 2.3640557 ]
152.8 0.180 0.271 1.394 -0.650 [1.79844341 0.37084309 1.74273    2.36405571]
110.9 0.074 0.312 0.533 -0.410 [0.74000875 0.42732382 0.66587648 1.49261247]
110.9 0.074 0.312 0.533 -0.410 [0.74000876 0.42732382 0.66587648 1.49261247]
110.9 0.074 0.312 0.533 -0.410 [0.74000875 0.42732383 0.66587648 1.49261247]
110.9 0.074 0.312 0.533 -0.410 [0.74000875 0.42732382 0.66587649 1.49261247]
110.9 0.074 0.312 0.533 -0.410 [0.74000875 0.42732382 0.66587648 1.49261248]
97.8 0.062 0.319 0.435 -0.383 [0.61993538 0.43652656 0.54412755 1.39386618]
97.8 0.062 0.319 0.435 -0.383 [0.61993539 0.43652656 0.54412755 1.39386618]
97.8 0.062 0.319 0.435 -0.383 [0.61993538 0.43652657 0.54412755 1.39386618]
97.8 0.062 0.319 0.435 -0.383 [0.61993538 0.43652656 0.54412756 1.39386618]
97.8 0.062 0.319 0.435 -0.383 [0.61993538 0.43652656 0.54412755 1.39386619]
151.

91.4 0.020 0.196 0.160 -0.262 [0.2        0.26880683 0.2        0.95093842]
91.4 0.020 0.196 0.160 -0.262 [0.20000001 0.26880683 0.2        0.95093842]
91.4 0.020 0.196 0.160 -0.262 [0.2        0.26880684 0.2        0.95093842]
91.4 0.020 0.196 0.160 -0.262 [0.2        0.26880683 0.20000001 0.95093842]
91.4 0.020 0.196 0.160 -0.262 [0.2        0.26880683 0.2        0.95093843]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26864005 0.2        0.9508291 ]
91.4 0.020 0.196 0.160 -0.261 [0.20000001 0.26864005 0.2        0.9508291 ]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26864006 0.2        0.9508291 ]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26864005 0.20000001 0.9508291 ]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26864005 0.2        0.95082911]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26862856 0.2        0.950785  ]
91.4 0.020 0.196 0.160 -0.261 [0.20000001 0.26862856 0.2        0.950785  ]
91.4 0.020 0.196 0.160 -0.261 [0.2        0.26862857 0.2        0.950785  ]
91.4 0.020 0

...

92.0 0.020 0.562 0.735 -0.422 [0.20000001 0.77005917 0.91865937 1.53561006]
92.0 0.020 0.562 0.735 -0.422 [0.2        0.77005918 0.91865937 1.53561006]
92.0 0.020 0.562 0.735 -0.422 [0.2        0.77005917 0.91865938 1.53561006]
92.0 0.020 0.562 0.735 -0.422 [0.2        0.77005917 0.91865937 1.53561007]
92.3 0.020 0.412 0.647 -0.344 [0.2        0.56417306 0.80918075 1.2516493 ]
92.3 0.020 0.412 0.647 -0.344 [0.20000001 0.56417306 0.80918075 1.2516493 ]
92.3 0.020 0.412 0.647 -0.344 [0.2        0.56417307 0.80918075 1.2516493 ]
92.3 0.020 0.412 0.647 -0.344 [0.2        0.56417306 0.80918076 1.2516493 ]
92.3 0.020 0.412 0.647 -0.344 [0.2        0.56417306 0.80918075 1.25164931]
91.9 0.020 0.519 0.710 -0.400 [0.2        0.71084745 0.88717392 1.45394449]
91.9 0.020 0.519 0.710 -0.400 [0.20000001 0.71084745 0.88717392 1.45394449]
91.9 0.020 0.519 0.710 -0.400 [0.2        0.71084746 0.88717392 1.45394449]
91.9 0.020 0.519 0.710 -0.400 [0.2        0.71084745 0.88717393 1.45394449]
91.9 0.020 0

91.5 0.020 0.201 0.298 -0.178 [0.2        0.27500556 0.37303316 0.64652472]
91.5 0.020 0.177 0.194 -0.187 [0.2        0.24224529 0.24306056 0.67847766]
91.5 0.020 0.177 0.194 -0.187 [0.20000001 0.24224529 0.24306056 0.67847766]
91.5 0.020 0.177 0.194 -0.187 [0.2        0.2422453  0.24306056 0.67847766]
91.5 0.020 0.177 0.194 -0.187 [0.2        0.24224529 0.24306057 0.67847766]
91.5 0.020 0.177 0.194 -0.187 [0.2        0.24224529 0.24306056 0.67847767]
91.4 0.020 0.173 0.160 -0.194 [0.2        0.23647306 0.2        0.70466459]
91.4 0.020 0.173 0.160 -0.194 [0.20000001 0.23647306 0.2        0.70466459]
91.4 0.020 0.173 0.160 -0.194 [0.2        0.23647307 0.2        0.70466459]
91.4 0.020 0.173 0.160 -0.194 [0.2        0.23647306 0.20000001 0.70466459]
91.4 0.020 0.173 0.160 -0.194 [0.2        0.23647306 0.2        0.7046646 ]
91.4 0.020 0.174 0.160 -0.195 [0.2        0.23784995 0.2        0.70865907]
91.4 0.020 0.174 0.160 -0.195 [0.20000001 0.23784995 0.2        0.70865907]
91.4 0.020 0

91.4 0.020 0.181 0.160 -0.219 [0.2        0.24842572 0.2        0.79546402]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24842571 0.20000001 0.79546402]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24842571 0.2        0.79546403]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24840399 0.2        0.79534509]
91.4 0.020 0.181 0.160 -0.219 [0.20000001 0.24840399 0.2        0.79534509]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.248404   0.2        0.79534509]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24840399 0.20000001 0.79534509]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24840399 0.2        0.7953451 ]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24831712 0.2        0.79486937]
91.4 0.020 0.181 0.160 -0.219 [0.20000001 0.24831712 0.2        0.79486937]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24831713 0.2        0.79486937]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24831712 0.20000001 0.79486937]
91.4 0.020 0.181 0.160 -0.219 [0.2        0.24831712 0.2        0.79486938]
91.4 0.020 0

91.6 0.020 0.200 0.370 -0.125 [0.2        0.27449692 0.46233635 0.45353419]
91.6 0.020 0.200 0.370 -0.125 [0.20000001 0.27449692 0.46233635 0.45353419]
91.6 0.020 0.200 0.370 -0.125 [0.2        0.27449693 0.46233635 0.45353419]
91.6 0.020 0.200 0.370 -0.125 [0.2        0.27449692 0.46233636 0.45353419]
91.6 0.020 0.200 0.370 -0.125 [0.2        0.27449692 0.46233635 0.4535342 ]
91.6 0.020 0.194 0.347 -0.126 [0.2        0.2659351  0.43345317 0.45779075]
91.6 0.020 0.194 0.347 -0.126 [0.20000001 0.2659351  0.43345317 0.45779075]
91.6 0.020 0.194 0.347 -0.126 [0.2        0.26593511 0.43345317 0.45779075]
91.6 0.020 0.194 0.347 -0.126 [0.2        0.2659351  0.43345318 0.45779075]
91.6 0.020 0.194 0.347 -0.126 [0.2        0.2659351  0.43345317 0.45779076]
91.6 0.020 0.185 0.311 -0.128 [0.2        0.25385418 0.38915432 0.46424638]
91.6 0.020 0.185 0.311 -0.128 [0.20000001 0.25385418 0.38915432 0.46424638]
91.6 0.020 0.185 0.311 -0.128 [0.2        0.25385419 0.38915432 0.46424638]
91.6 0.020 0

KeyboardInterrupt: 

In [ ]:
# Possibly reasonable values
# 91.8 0.050 0.365 0.445 -0.354 [0.5        0.5        0.55624399 1.28887181]

In [55]:
# Let's assign these values
x = [0.5     ,   0.5     ,   0.55624399, 1.28887181]

model_params['Wake Turbulence Parameters']['ti_initial'] = x[0] * 0.1 # how much to scale inital by
model_params['Wake Turbulence Parameters']['ti_constant'] = x[1] * 0.73 # how much to scale inital by
model_params['Wake Turbulence Parameters']['ti_ai'] = x[2] * 0.8 # how much to scale inital by
model_params['Wake Turbulence Parameters']['ti_downstream'] = x[3] * -0.275 # how much to scale inital by

fi_tune.set_model_parameters(model_params)

fi_dict['gt'] = (fi_tune,'r','d','gauss_tune')

Wake velocity parameter alpha set to 0.58
Wake velocity parameter beta set to 0.077
Wake velocity parameter calculate_VW_velocities set to True
Wake velocity parameter eps_gain set to 0.3
Wake velocity parameter ka set to 0.22077999999999998
Wake velocity parameter kb set to 0.002
Wake velocity parameter use_yaw_added_recovery set to True
Wake velocity parameter yaw_recovery_alpha set to 0.03
Wake deflection parameter ad set to 0.0
Wake deflection parameter alpha set to 0.58
Wake deflection parameter bd set to 0.0
Wake deflection parameter beta set to 0.077
Wake deflection parameter eps_gain set to 0.3
Wake deflection parameter ka set to 0.38
Wake deflection parameter kb set to 0.004
Wake deflection parameter use_secondary_steering set to True
Wake turbulence parameter ti_ai set to 0.44499519200000004
Wake turbulence parameter ti_constant set to 0.365
Wake turbulence parameter ti_downstream set to -0.35443974775000003
Wake turbulence parameter ti_initial set to 0.05


In [56]:
# Save this new floris dictionary
fi_dict


{'g': (<floris.tools.floris_interface.FlorisInterface at 0x10790eb10>,
  'b',
  's',
  'gauss'),
 'gt': (<floris.tools.floris_interface.FlorisInterface at 0x119373890>,
  'r',
  'd',
  'gauss_tune'),
 'gch': (<floris.tools.floris_interface.FlorisInterface at 0x119373a10>,
  'g',
  'o',
  'gch_paper'),
 'leg': (<floris.tools.floris_interface.FlorisInterface at 0x11953a490>,
  'gray',
  '^',
  'legacy')}

In [57]:
pickle.dump( fi_dict, open( "floris_models_3.p", "wb" ) )